In [ ]:
import numpy as np
import pandas as pd
import copy
import pickle
import matplotlib.pyplot as plt
from scipy import optimize
import time
plt.style.use('seaborn')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from simulation_util import *


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#파일 저장 테스트코드
#실행 후, 해당 폴더에 빈파일이 생성되는지 체크.
#with open(f'./drive/MyDrive/krxdata/빈파일.pkl', 'wb') as f:
#    pickle.dump([0], f)

def prep(data:pd.DataFrame, coliloc=7, t='Int64') -> pd.DataFrame:
    data = copy.deepcopy(data)
    data.columns = data.iloc[coliloc]
    data.index.name = 'date'
    data.columns.name = 'ticker'
    return data.iloc[13:].astype(t) #if Int64, I MUST BE Capital


with open('./data/tradingvalue.pkl', 'rb') as f:
    tamt_ = pickle.load(f)
with open('./data/modifiedstockprice.pkl', 'rb') as f:
    closeprice_ = pickle.load(f)
with open('./data/marketcap.pkl', 'rb') as f:
    cap_ = pickle.load(f)
indices_ = pd.read_excel('./data/domesticindexprice.xlsx' , index_col=0)



closeprice = prep(closeprice_)
tamt = prep(tamt_, t='float32')
indices = prep(indices_, coliloc=7, t='float32')
cap = prep(cap_, t='float32') 

daily_ret = closeprice.pct_change(1).iloc[1:]
tamt = tamt.iloc[1:]

#티커
tickers, names= closeprice_.iloc[7:9].values    

#Hyperparameters
learning_rate = 0.000055
epoch = 150
AElookback = 180 # 60 * 3 9개월? 오토인코더 룩백인감
lookback = 60

#parameter space
t_space = [0.3, 0.4, 0.5, 0.6] #운용역 재량권
M_space = indices.columns
print(list(M_space))

M='KRX 300'
sim = Simulation(tamt, closeprice, cap, indices, lookback, AElookback, learning_rate, epoch)

#result1 = simulate(t=0.3, M=M, target_corr=0.75, AE=True )
#save(data = result1, t = 0.3, M=M)

#result2 = simulate(t=0.4, M=M, target_corr=0.75, AE=True )
#save(data = result2, t = 0.4, M=M)

result3 = sim.simulate(t=0.5, M=M, target_corr=0.75, AE=True )
sim.save(data = result3, t = 0.5, M=M)

result4 = sim.simulate(t=0.6, M=M, target_corr=0.75, AE=True )
sim.save(data = result4, t = 0.6, M=M)


res = sim.report('KRX 300', 0.6)